In [1]:
!java -version
!python --version

openjdk version "11.0.12" 2021-07-20
OpenJDK Runtime Environment Microsoft-25199 (build 11.0.12+7)
OpenJDK 64-Bit Server VM Microsoft-25199 (build 11.0.12+7, mixed mode)


Python 3.9.12


In [2]:
!pip install pyspark

     -------------------------------------- 281.4/281.4 MB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ------------------------------------- 199.7/199.7 kB 12.6 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824045 sha256=70fbe84f71e4fc1eb0485f6f3d4684b68d9dae7dba07a3dc526c91f9b48dac55
  Stored in directory: c:\users\sansk\appdata\local\pip\cache\wheels\8e\1f\b7\ed748602b39b0e85d8d322ca481856faea8d5360900ee84b00
Successfully built pyspark


In [21]:
import pyspark
import findspark

In [22]:
findspark.init()
findspark.find()

'E:\\spark-3.3.1-bin-hadoop2'

In [23]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession

In [27]:
conf = pyspark.SparkConf().setAppName('SparkApp').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)

In [28]:
from pyspark.sql.types import Row
from datetime import datetime

In [29]:
simple_data = sc.parallelize([1,23,56])
simple_data

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [30]:
simple_data.count()
simple_data.first()

3

In [32]:
simple_data.collect()

[1, 23, 56]

In [33]:
value = simple_data.map(lambda x: x*x*x)
value.collect()

[1, 12167, 175616]

In [51]:
sample = [[1,"James",34],[2,"Logan",45]]
simple_list= sc.parallelize(sample)
print(simple_list.count())
simple_list.first()

2


[1, 'James', 34]

In [52]:
simple_list.collect()
cols = ['S.no','Name','Age']

In [72]:
records = simple_list.toDF(cols)
records.show()

+----+-----+---+
|S.no| Name|Age|
+----+-----+---+
|   1|James| 34|
|   2|Logan| 45|
+----+-----+---+



In [77]:
records.createTempView("Record")

In [78]:
spark.sql('select * from record where age>35').show()


+----+-----+---+
|S.no| Name|Age|
+----+-----+---+
|   2|Logan| 45|
+----+-----+---+



In [64]:
complex_data = sc.parallelize([Row(col_string = "James", col_double = 3.14,col_int = 34, col_bool = True,col_list=[12,"Lo",45]),
                              Row(col_string = "Phil", col_double = 9.14,col_int = 39, col_bool = False, col_list=[12,"Ho",49])
                              ])
complex_data_df=complex_data.toDF()
complex_data_df.printSchema()

root
 |-- col_string: string (nullable = true)
 |-- col_double: double (nullable = true)
 |-- col_int: long (nullable = true)
 |-- col_bool: boolean (nullable = true)
 |-- col_list: array (nullable = true)
 |    |-- element: long (containsNull = true)



In [66]:
complex_data_df.show()

+----------+----------+-------+--------+--------------+
|col_string|col_double|col_int|col_bool|      col_list|
+----------+----------+-------+--------+--------------+
|     James|      3.14|     34|    true|[12, null, 45]|
|      Phil|      9.14|     39|   false|[12, null, 49]|
+----------+----------+-------+--------+--------------+



In [67]:
sql = SQLContext(sc)
sql

E:\Python\lib\site-packages\pyspark\sql\context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [68]:
# forms a dataframe directly
df= sql.range(4)
df

DataFrame[id: bigint]

In [69]:
df.show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
+---+



In [70]:
data = [[1,"James",34],[2,"Logan",45]]
sql.createDataFrame(data,['Sno','Name','Age']).show()

+---+-----+---+
|Sno| Name|Age|
+---+-----+---+
|  1|James| 34|
|  2|Logan| 45|
+---+-----+---+



In [79]:
data2 = sc.parallelize([Row(1,"HP",5),
                       Row(2,"Azus",4),
                       Row(3,"Dell",3)])

In [81]:
data2.toDF().show()

+---+----+---+
| _1|  _2| _3|
+---+----+---+
|  1|  HP|  5|
|  2|Azus|  4|
|  3|Dell|  3|
+---+----+---+



In [82]:
col_name = Row('Sno','Laptop','Warranty')
laps = data2.map(lambda r: col_name(*r))
laps

PythonRDD[99] at RDD at PythonRDD.scala:53

In [84]:
laps.collect()

[Row(Sno=1, Laptop='HP', Warranty=5),
 Row(Sno=2, Laptop='Azus', Warranty=4),
 Row(Sno=3, Laptop='Dell', Warranty=3)]

In [85]:
laps_df = sql.createDataFrame(laps)
laps_df.show()

+---+------+--------+
|Sno|Laptop|Warranty|
+---+------+--------+
|  1|    HP|       5|
|  2|  Azus|       4|
|  3|  Dell|       3|
+---+------+--------+



In [86]:
laps_df.first()

Row(Sno=1, Laptop='HP', Warranty=5)

In [92]:
# Accessing columns using select
laps_df.select("Laptop","Warranty").show()

+------+--------+
|Laptop|Warranty|
+------+--------+
|    HP|       5|
|  Azus|       4|
|  Dell|       3|
+------+--------+



In [96]:
laps_df.select("Sno","Warranty").withColumn("New Lap",laps_df.Sno+laps_df.Warranty).show()

+---+--------+-------+
|Sno|Warranty|New Lap|
+---+--------+-------+
|  1|       5|      6|
|  2|       4|      6|
|  3|       3|      6|
+---+--------+-------+



In [89]:
cell=laps_df.collect()[0][2]
cell

5

In [91]:
# Accessing columns using .rdd property
laps_df.rdd.map(lambda x: (x.Laptop, x.Warranty)).collect()

[('HP', 5), ('Azus', 4), ('Dell', 3)]

In [93]:
laps_df.rdd.map(lambda x: (x.Laptop+" 2018")).collect()

['HP 2018', 'Azus 2018', 'Dell 2018']

In [97]:
laps_df.withColumnRenamed("Warranty","Guarantee").show() # temporary

+---+------+---------+
|Sno|Laptop|Guarantee|
+---+------+---------+
|  1|    HP|        5|
|  2|  Azus|        4|
|  3|  Dell|        3|
+---+------+---------+



In [98]:
laps_df.show()

+---+------+--------+
|Sno|Laptop|Warranty|
+---+------+--------+
|  1|    HP|       5|
|  2|  Azus|       4|
|  3|  Dell|       3|
+---+------+--------+



In [99]:
laps_df.select(laps_df.Warranty.alias("Guarantee")).show()

+---------+
|Guarantee|
+---------+
|        5|
|        4|
|        3|
+---------+



In [100]:
# Pandas dataframes are quite similar to spark dataframes
# converting a spark dataframe to pandas dataframe and vice versa

import pandas

In [101]:
df_pandas = laps_df.toPandas()
df_pandas

,Sno,Laptop,Warranty
0,1,HP,5
1,2,Azus,4
2,3,Dell,3


In [103]:
df_spark=sql.createDataFrame(df_pandas).show()

+---+------+--------+
|Sno|Laptop|Warranty|
+---+------+--------+
|  1|    HP|       5|
|  2|  Azus|       4|
|  3|  Dell|       3|
+---+------+--------+

